In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from architectures import *
from torch.utils.tensorboard import SummaryWriter

In [4]:
writer = SummaryWriter()

for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)
writer.flush()

In [2]:
resnet = DeepSetNet()
S,B,C,H,W = 11, 2, 3, 256, 256
x = torch.rand(B, S, C, H, W)
resnet(x).size()

torch.Size([2, 3, 256, 256])

In [16]:
batch_size, features, length = 256**2, 64, 10 
multihead_attn = nn.MultiheadAttention(64,8).cuda()
x = torch.rand(length,batch_size, features).cuda()
attn_output, attn_output_weights = multihead_attn(x,x,x)


(attn_output_weights > 0).all()

tensor(True, device='cuda:0')

In [4]:
S,B,C,H,W = 11, 2, 64, 256, 256
xs = [torch.zeros(B,C,H,W) for _ in range(S)]
agg = AttentionAggregation(embed_dim=C)
xs = agg(xs)
xs.size()

torch.Size([11, 2, 64, 256, 256])

In [9]:
x = torch.rand(10,64,256,256)
softm = nn.Softmax2d()
x = softm(x)
x.sum(1)

tensor([[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

        [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

        [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.